# Credmark Modeling Framework Example for Jupyter notebook
## Ledger Model

version: 2022.6.3

In [ ]:
from credmark.cmf.engine.model_loader import ModelLoader
from credmark.cmf.engine.context import EngineModelContext
from credmark.cmf.model.context import ModelContext

from credmark.dto import EmptyInput

from credmark.cmf.types import Contract, Token, Price, Address, Account, Contracts, Tokens, Accounts, BlockNumber

from credmark.cmf.types.ledger import (
    BlockTable, ContractTable,
    LogTable, ReceiptTable, TokenTable, TokenTransferTable,
    TraceTable, TransactionTable, LedgerTable,
    LedgerAggregate, LedgerModelOutput,
    ContractLedger
)

from credmark.cmf.engine.dev_models.console import get_dt, get_block, log_output

## Initialize

In [ ]:
model_loader = ModelLoader(['../models'], None, True)

params = {'chain_id': 1,
          'block_number': None,
          'model_loader': model_loader,
          'chain_to_provider_url': {'1': 'http://localhost:10444'},
          'api_url': 'http://localhost:8700',
          'run_id': None,
          'console': True,
          'use_local_models': None
         }
context = EngineModelContext.create_context(**params)

ledger = context.ledger
run_model = context.run_model
models = context.models
block_number = context.block_number
chain_id = context.chain_id
web3 = context.web3
run_model_historical = context.historical.run_model_historical
run_model_historical_blocks = context.historical.run_model_historical_blocks

## Example

### Contract - Uni V2

In [ ]:
contract = Contract(address='0x0d4a11d5EEaaC28EC3F61d100daF4d40471f1852')
print(contract.functions.name().call(), contract.functions.symbol().call())

for addr in [contract.functions.token0().call(), contract.functions.token1().call()]:
    print(Token(address=addr).symbol)

### Ledger Examples

- Get the max value group by `to`

In [ ]:
contract = Contract(address='0x0d4a11d5EEaaC28EC3F61d100daF4d40471f1852')

ret = (
    contract.ledger.functions.Transfer(
        columns=[ContractLedger.Functions.InputCol('to'),
                 ContractLedger.Functions.Columns.TXN_BLOCK_NUMBER],
        aggregates=[
            ContractLedger.Aggregate(
            f'MAX({ContractLedger.Functions.InputCol("value")})', 'max_value')
        ],
        group_by=f"{ContractLedger.Functions.InputCol('to')},{ContractLedger.Functions.Columns.TXN_BLOCK_NUMBER}",
        order_by='"max_value" desc',
        where=f'{ContractLedger.Functions.Columns.TXN_BLOCK_NUMBER} > {context.block_number - 2000000}',
        limit='10'
    ).to_dataframe())
ret

- Get max amount0Out by `to`.

In [ ]:
ret = (
    contract.ledger.events.Swap(
        columns=[ContractLedger.Events.InputCol('to')],
        aggregates=[
                ContractLedger.Aggregate(
                    f'MAX({ContractLedger.Events.InputCol("amount0Out")})', 'max_value')
            ],
        group_by=ContractLedger.Events.InputCol('to'),
        order_by='"max_value" desc',
        where=f'{ContractLedger.Events.Columns.EVT_BLOCK_NUMBER} > {context.block_number - 2000000}',
        limit='5')
    .to_dataframe())
ret

- Get max amount0Out and max amount1Out

In [ ]:
contract = Contract(address='0x0d4a11d5EEaaC28EC3F61d100daF4d40471f1852')
ret = (
    contract.ledger.events.swap(
        columns=[],
        aggregates=[
            ContractLedger.Aggregate(
                f'MAX({ContractLedger.Events.InputCol("amount0Out")})', 'max_amount0Out'),
            ContractLedger.Aggregate(
                f'MAX({ContractLedger.Events.InputCol("amount1Out")})', 'max_amount1Out')
        ]
    ).to_dataframe())
ret

- Get the transaction hash containing the max amount0Out.

In [ ]:
contract = Contract(address='0x0d4a11d5EEaaC28EC3F61d100daF4d40471f1852')
ret2 = (
    contract.ledger.events.Swap(
        columns=[ContractLedger.Events.Columns.EVT_HASH],
        aggregates=[
            ContractLedger.Aggregate(
                f'MAX({ContractLedger.Events.InputCol("amount0Out")})', 'max_amount0Out')
        ],
        group_by=','.join([ContractLedger.Events.Columns.EVT_HASH]),
        having=f'max({ContractLedger.Events.InputCol("amount0Out")}) >= {ret.max_amount0Out[0]}',
    ).to_dataframe())
ret2

- Get the transaction hash containing the max amount0Out or max amount1Out

For any column in `having` needs to be in `aggregates`.

In [ ]:
contract = Contract(address='0x0d4a11d5EEaaC28EC3F61d100daF4d40471f1852')
ret2 = (
    contract.ledger.events.Swap(
        columns=[ContractLedger.Events.Columns.EVT_HASH],
        aggregates=[
            ContractLedger.Aggregate(
                f'MAX({ContractLedger.Events.InputCol("amount0Out")})', 'max_amount0Out'),
            ContractLedger.Aggregate(
                f'MAX({ContractLedger.Events.InputCol("amount1Out")})', 'max_amount1Out')
        ],
        group_by=','.join([ContractLedger.Events.Columns.EVT_HASH]),
        having=(f'max({ContractLedger.Events.InputCol("amount0Out")}) >= {ret.max_amount0Out[0]} or '
                f'max({ContractLedger.Events.InputCol("amount1Out")}) >= {ret.max_amount1Out[0]}'),
    ).to_dataframe())
ret2